# Desarrollo del Proyecto 1 - Recomendación de películas
Hecho por: Alejandra Salas

### Extract_Transform_Load (ETL)

## 1. Extraer los datos de los archivos

In [65]:
# Se importan las librerías a usar.
import numpy as np
import pandas as pd
import json
import ast

In [66]:
# Cargamos los datos que se encuentran en archivo tipo ".csv" a un dataframe.
creditos = pd.read_csv('credits.csv',sep=',')
movies =pd.read_csv('movies_dataset.csv', sep=',',low_memory=False)

In [67]:
# Visualizamos los datos del dataframe "creditos".
creditos.sample(2)

,cast,crew,id
13501,"[{'cast_id': 1, 'character': 'Ray Koval', 'cre...","[{'credit_id': '52fe46db9251416c75085cf9', 'de...",16558
21954,"[{'cast_id': 1, 'character': 'Marcus', 'credit...","[{'credit_id': '591b1652925141061e066eab', 'de...",44745


In [68]:
# Visualizamos los datos del dataframe "movies".
movies.sample(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19171,False,NaN,0,[],NaN,60003,tt0345852,en,Red Rover,Two siblings played by William Baldin and Jodi...,...,2006-10-08,0.0,103.0,[],Released,A family. A legacy. A deadly curse.,Red Rover,False,4.7,3.0
617,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",NaN,28121,tt0117891,en,A Thin Line Between Love and Hate,Nightclub manager Darnell Wright is a perpetua...,...,1996-04-04,0.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"When you play, you pay.",A Thin Line Between Love and Hate,False,5.7,15.0


##### _Observaciones_
Se observa que en las columnas "cast" y "crew" del dataframe "creditos", se encuentran anidadas en un formato diccionario. Así mismo, las columnas "belongs_to_collection", "genres", "production_companies", "production_countries" y "spoken_languages" del datraframe "movies", también se encuentran anidados en formato diccionario.

In [69]:
# Revisamos la información del dataframe "creditos" para conocer si cuneta con valores nulos.
creditos.info()
# Así mismo se puede observar los tipos de datos que contiene cada columna.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [70]:
# Revisamos la información del dataframe "movies" para conocer si cuneta con valores nulos.
movies.info()
# Si bien hay algunos datos nulos, no sería factible eliminarlos ya que podemos tener pocos datos a analizar.
# Así mismo se puede observar los tipos de datos que contiene cada columna.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [71]:
# Visualizamos la forma de la estructura del dataframe "creditos".
creditos.shape
# Tiene 45476 filas y 3 columnas.

(45476, 3)

In [72]:
# Visualizamos la forma de la estructura del dataframe "movies".
movies.shape
# Tiene 45466 filas y 24 columnas.

(45466, 24)

## 2. Transformaciones de los dataframes

Algunos campos, como "cast" y "crew" del dataframe "creditos" y  "belongs_to_collection", "genres", "production_companies", "production_countries" y "spoken_languages" del datraframe "movies" están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila. Se desanidarán para poder utilizarlos y se unirán al dataset de nuevo para hacer alguna de las consultas de la API.

*Crew*

In [73]:
# Observamos el contenido de la columna "crew" del dataframe "creditos", en este contenido, necesitamos el nombre del "Director".
creditos['crew'].head(3).values

array(['[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name

In [74]:
# Extraemos el nombre del director de la columna "crew" del dataframe "creditos"
def extract_director(row):
    try:
        crew_list = ast.literal_eval(row['crew'])
        for crew_member in crew_list:
            if crew_member.get('job') == 'Director':
                return crew_member.get('name')
    except (SyntaxError, ValueError):
        pass
    return None

# Aplicamos la función para obtener el nombre del director
creditos['Director'] = creditos.apply(extract_director, axis=1)

creditos.head(5)


,cast,crew,id,Director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [75]:
# Comprobamos si los datos extraidos son correctos con los datos originales.
creditos['crew'].head(10).values

array(['[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name

In [76]:
creditos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45476 non-null  object
 1   crew      45476 non-null  object
 2   id        45476 non-null  int64 
 3   Director  44589 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


*Belongs_to_collection*

In [77]:
# Utilizamos ast.literal_eval para evaluar la columna 'belongs_to_collection' como un diccionario de Python
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')

# Extraemos el nombre de la colección
movies['collection_name'] = movies['belongs_to_collection'].apply(lambda x: x.get('name', '') if isinstance(x, dict) else 'None')

# Imprimimos movies con la columna normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
24634,False,None,0,"[{'id': 18, 'name': 'Drama'}]",NaN,289314,tt3012698,en,Olive Kitteridge,The story focuses on a middle-school math teac...,...,0.0,233.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There's no such thing as a simple life.,Olive Kitteridge,False,7.6,66.0,None
44304,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,249233,tt3499286,en,Anjelah Johnson: The Homecoming Show,Former Raiders cheerleader turned stand-up com...,...,0.0,60.0,[],Released,NaN,Anjelah Johnson: The Homecoming Show,False,4.0,1.0,None
41596,False,None,7000000,"[{'id': 18, 'name': 'Drama'}]",http://www.20thcenturywomen-movie.com/,342737,tt4385888,en,20th Century Women,"In 1979 Santa Barbara, California, Dorothea Fi...",...,5816211.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,20th Century Women,False,7.3,213.0,None
15001,False,None,0,"[{'id': 18, 'name': 'Drama'}]",http://www.kuuki-ningyo.com,25053,tt1371630,ja,空気人形,"An ""air doll"" suddenly come to life one day. W...",...,0.0,125.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Air Doll,False,6.6,53.0,None
8922,False,None,25000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 27, 'na...",NaN,41967,tt0096218,en,Tales from the Gimli Hospital,While their mother is dying in the modern Giml...,...,0.0,68.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,It all happened in a Gimli we no longer know.,Tales from the Gimli Hospital,False,7.4,6.0,None


*Genres*

In [78]:
# Utilizamos ast.literal_eval para evaluar la columna 'genres' como un diccionario de Python
movies['genres'] = movies['genres'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')
        
# Extraemos los nombres de los géneros con una función
def extract_genre_names(genres_list):
    genre_names = [genre['name'] for genre in genres_list]
    return ', '.join(genre_names)

# Aplicamos la función a cada elemento de la columna 'genres'
movies['genre'] = movies['genres'].apply(extract_genre_names)

# Imprimimos el DataFrame con la columna de géneros normalizados
movies.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,genre
507,False,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,25557,tt0111003,en,The Puppet Masters,"The Earth is invaded by alien parasites aka ""s...",...,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Trust No One,The Puppet Masters,False,5.6,41.0,None,"Horror, Science Fiction, Thriller"
43372,False,None,950000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",NaN,30593,tt0070165,en,Heavy Traffic,"An ""underground"" cartoonist contends with life...",...,76.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,More Spice from the makers of Fritz the Cat!,Heavy Traffic,False,6.6,18.0,None,"Animation, Comedy, Drama"
276,False,None,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,8438,tt0113870,en,Murder in the First,Inspired by a true story. A petty criminal sen...,...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The trial that brought down Alcatraz,Murder in the First,False,6.9,140.0,None,"Crime, Drama"
34383,False,None,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 16, 'na...",NaN,65592,tt0229664,fr,Princes et princesses,This is a very fantastic and different world a...,...,70.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Princes and Princesses,False,7.6,25.0,None,"Fantasy, Animation, Drama, Science Fiction, Ro..."
18523,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,79133,tt2114375,en,"Dylan Moran: Yeah, Yeah","Dylan Moran, star of Black Books, Shaun of the...",...,73.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,"Dylan Moran: Yeah, Yeah",False,7.5,9.0,None,Comedy


In [79]:
# Cambiamos los campos vacíos de las columnas "genres"  y "genre" por "None" en el daraframe "movies".
movies['genres'] = movies['genres'].apply(lambda x: 'None' if x == [] else x)
movies['genre'] = movies['genre'].apply(lambda x: 'None' if x == '' else x)

In [80]:
movies['genres'][55]

'None'

In [81]:
# Comprobamos que estos valores estén con la palabra "None".
movies.sample(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,genre
39234,False,None,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 10770,...",http://www.mylifetime.com/movies/mother-may-i-...,399747,tt5480528,en,"Mother, May I Sleep with Danger?","When theater major Leah, brings home the speci...",...,86.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,"Mother, May I Sleep with Danger?",False,5.4,15.0,None,"Thriller, TV Movie"
26794,False,None,0,"[{'id': 18, 'name': 'Drama'}]",NaN,129251,tt0099842,en,The Incident,"Small town lawyer, Harmon Cobb, defends a Nazi...",...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Small Town,The Incident,False,4.8,4.0,None,Drama
14860,False,None,14000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,12273,tt1146325,hi,Singh Is Kinng,"A comic caper about Happy Singh, a Punjabi vil...",...,135.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}, {'iso_...",Released,Summer 2008: The gang of Punjab arrives with a...,Singh Is Kinng,False,5.3,22.0,None,"Action, Comedy"


*Production_companies*

In [82]:
# Realizamos una función para normalizar la columna 'production_companies'
def normalize_production_companies(companies_list):
    if isinstance(companies_list, list):  # Si ya es una lista, asumimos que los datos están correctamente formateados
        company_names = [company['name'] for company in companies_list]
        return ', '.join(company_names)
    else:  # Si no es una lista, utilizamos ast.literal_eval para convertir la cadena JSON en una lista
        try:
            companies_list = ast.literal_eval(companies_list)
            if isinstance(companies_list, list):
                company_names = [company['name'] for company in companies_list]
                return ', '.join(company_names)
            else:
                return 'None'
        except (SyntaxError, ValueError):
            return 'None'

# Aplicamos la función a cada elemento de la columna 'production_companies'
movies['company'] = movies['production_companies'].apply(normalize_production_companies)

# Imprimimos movies con la columna de compañías de producción normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,genre,company
41811,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,308919,tt3627780,en,Wishin' and Hopin',Based on the New York Times best-selling novel...,...,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Wishin' and Hopin',False,4.0,1.0,None,"Drama, Comedy","Synthetic Cinema International, Flavorlab, Sta..."
26311,False,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,107999,tt0040187,en,The Bride Goes Wild,McGrath publishes books for children and Uncle...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Bride Goes Wild,False,6.0,1.0,None,"Comedy, Romance",Metro-Goldwyn-Mayer (MGM)
22979,False,None,145000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",NaN,82703,tt0864835,en,Mr. Peabody & Sherman,"A young boy and his dog, who happens to have a...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He's Leaving His Mark On History,Mr. Peabody & Sherman,False,6.7,861.0,None,"Animation, Adventure, Family","Pacific Data Images (PDI), DreamWorks Animatio..."
34872,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,172443,tt0006809,en,His Picture in the Papers,A young man can only get the woman he loves if...,...,[],Released,NaN,His Picture in the Papers,False,4.0,1.0,None,Comedy,Fine Arts Film Company
15587,False,None,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",NaN,61430,tt0044331,en,Affair in Trinidad,A nightclub singer enlists her brother-in-law ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"""You weren't the first... and you won't be the...",Affair in Trinidad,False,6.3,5.0,None,"Thriller, Mystery, Romance","Columbia Pictures Corporation, The Beckworth C..."


In [83]:
# Cambiamos los campos vacíos de las columnas "genres"  y "genre" por "None" en el daraframe "movies".
movies['production_companies']= movies['production_companies'].apply(lambda x: 'None' if x == '[]' else x)
movies['company'] = movies['company'].apply(lambda x: 'None' if x == '' else x)

*Production_countries*

In [84]:
# Aseguramos que la columna 'production_countries' contenga datos JSON válidos
movies['production_countries'] = movies['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')

# Realizamos una función para normalizar la columna 'production_countries'
def normalize_production_countries(countries_list):
    if isinstance(countries_list, list):
        country_names = [country['name'] for country in countries_list]
        return ', '.join(country_names)
    else:
        return 'None'

# Aplicamos la función a cada elemento de la columna 'production_countries'
movies['country'] = movies['production_countries'].apply(normalize_production_countries)

# Imprimimos movies con la columna de países de producción normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,collection_name,genre,company,country
22866,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,77458,tt1735200,fr,Le cochon de Gaza,"After a tempest, fishermen do not find only fi...",...,Released,NaN,When Pigs Have Wings,False,7.1,27.0,None,Comedy,StudioCanal,"France, Germany, Belgium"
19359,False,None,17000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",http://theapparition.warnerbros.com,79694,tt1433822,en,The Apparition,Plagued by frightening occurrences in their ho...,...,Released,Once You Believe You Die,The Apparition,False,4.5,140.0,None,"Horror, Thriller","StudioCanal, Dark Castle Entertainment, Warner...",United States of America
11047,False,None,0,"[{'id': 18, 'name': 'Drama'}]",NaN,17741,tt0426627,en,Stoned,A chronicle of the sordid life and suspicious ...,...,Released,The story of the original Rolling Stone,Stoned,False,5.4,17.0,None,Drama,"Wildgaze Films, Number 9 Films, Finola Dwyer P...",United Kingdom
5358,False,None,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,43915,tt0061653,en,Fathom,"While touring abroad in Europe, beautiful Amer...",...,Released,The world's most uncovered undercover agent!,Fathom,False,7.1,7.0,None,"Action, Comedy, Drama",Twentieth Century Fox Film Corporation,United Kingdom
11372,False,"{'id': 52749, 'name': 'The Butterfly Effect Co...",0,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",http://www.newline.com/properties/butterflyeff...,14620,tt0457297,en,The Butterfly Effect 2,"After his girlfriend, Julie and two best frien...",...,Released,Can you change your past without destroying yo...,The Butterfly Effect 2,False,4.6,255.0,The Butterfly Effect Collection,"Drama, Science Fiction, Thriller","New Line Cinema, FilmEngine, BenderSpink",United States of America


In [85]:
movies['production_countries']= movies['production_countries'].apply(lambda x: 'None' if x == [] else x)
movies['country'] = movies['country'].apply(lambda x: 'None' if x == '' else x)

*Spoken_languages*

In [86]:
# Aseguramos que la columna 'spoken_languages' contenga datos JSON válidos
movies['spoken_languages'] = movies['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else 'None')

# Realizamos una función para normalizar la columna "spoken_languages"
def normalize_spoken_languages(languages_list):
    if isinstance(languages_list, list):
        language_names = [language['name'] for language in languages_list]
        return ', '.join(language_names)
    else:
        return 'None'

# Aplicamos la función a cada elemento de la columna 'spoken_languages'
movies['spoken_language'] = movies['spoken_languages'].apply(normalize_spoken_languages)

# Imprimimos movies con la columna de lenguajes normalizada
movies.sample(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,collection_name,genre,company,country,spoken_language
39466,False,"{'id': 53043, 'name': 'The Magick Lantern Cycl...",0,"[{'id': 18, 'name': 'Drama'}]",NaN,46784,tt0042872,en,Rabbit's Moon,Shot in the Films du Panthéon Studio in Paris ...,...,NaN,Rabbit's Moon,False,7.0,10.0,The Magick Lantern Cycle,Drama,None,United States of America,English
26039,False,"{'id': 135489, 'name': 'Wishmaster Collection'...",0,"[{'id': 53, 'name': 'Thriller'}, {'id': 27, 'n...",NaN,28635,tt0254871,en,Wishmaster 3: Beyond the Gates of Hell,"The evil Djinn is back at again, this time wre...",...,"Three wishes, one nightmare.",Wishmaster 3: Beyond the Gates of Hell,False,3.8,38.0,Wishmaster Collection,"Thriller, Horror","Overseas FilmGroup, GFT Entertainment, Artisan...","Canada, United States of America",English
19251,False,None,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 53, '...",http://www.hierrolapelicula.com/,38256,tt1318025,es,Hierro,Traveling on a ferry heading to the island of ...,...,A travel. A search. A mystery,Hierro,False,6.1,21.0,None,"Mystery, Thriller","Telecinco Cinema, Madrugada Films",Spain,"Français, Español"
37462,False,None,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,347669,tt3169832,es,Incidencias,The last train travel of the year from Barcelo...,...,NaN,Stranded,False,5.3,14.0,None,Comedy,"Castelao Pictures, Somnium Films",Spain,Español
21223,False,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",NaN,132064,tt1711018,en,Vile,A group of attractive young people (of course)...,...,Give Till It Hurts.,Vile,False,4.8,51.0,None,"Horror, Thriller",Tony-Seven Films,United States of America,English


In [87]:
movies['spoken_languages']= movies['spoken_languages'].apply(lambda x: 'None' if x == [] else x)
movies['spoken_language']= movies['spoken_language'].apply(lambda x: 'None' if x == '' else x)

## 3. Eliminamos las columnas que no están desanidadas y las que no nos servirán

In [88]:
creditos.drop(columns=['cast','crew'],inplace=True)
creditos.head()

,id,Director
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [89]:
movies.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage',], inplace = True)
movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,genre,company,country,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français"
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America,English
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,None,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America,English


## 4. Duplicamos valores para mejorar la consulta de desanidados

##### _Observaciones_
En las columnas desanidas del dataframe "movies", hay algunos valores separados por comas, por ejemplo "genres" que contiene (Animation, Comedy, Family).
Es por eso que haremos un análsis con valores únicos y posteriormente duplicaremos las filas para que de desagreguen esos valores de cada columna.

Imprimimos los datos únicos de los datos desanidados y la cantidad de estos

In [90]:
# Al imprimir los valores únicos, nos damos cuenta que hay valores únicos que contienen a partir de 2 a más características, como lo explicado en genres anteriormente.
print (movies.collection_name.unique())
print(movies.genre.unique())
print(movies.company.unique())
print(movies.country.unique())
print(movies.spoken_language.unique())


['Toy Story Collection' 'None' 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']
['Animation, Comedy, Family' 'Adventure, Fantasy, Family'
 'Romance, Comedy' ... 'Crime, Comedy, Action, Family'
 'Action, Mystery, Thriller, Horror' 'Family, Animation, Romance, Comedy']
['Pixar Animation Studios'
 'TriStar Pictures, Teitler Film, Interscope Communications'
 'Warner Bros., Lancaster Gate' ...
 'Westdeutscher Rundfunk (WDR), Working Title Films, 20th Century Fox Television, CanWest Global Communications'
 'Sine Olivia' 'Yermoliev']
['United States of America' 'Germany, United States of America'
 'United Kingdom, United States of America' ...
 'Poland, Czech Republic, Slovakia' 'Cuba, Germany, Spain'
 'Egypt, Italy, United States of America']
['English' 'English, Français' 'English, Español' ... 'svenska, Deutsch'
 'العربية, Polski' 'Fulfulde, English']


In [91]:
# Imprimiremos la cantidad de valores únicos para ver el cambio posteriormente al hacer el duplicado.
print ('Cantidad de collection_name:',movies.collection_name.nunique())
print('Cantidad de genre:',movies.genre.nunique())
print('Cantidad de company:',movies.company.nunique())
print('Cantidad de country:',movies.country.nunique())
print('Cantidad de spoken_language:',movies.spoken_language.nunique())

Cantidad de collection_name: 1696
Cantidad de genre: 4069
Cantidad de company: 22671
Cantidad de country: 2390
Cantidad de spoken_language: 1842


Crear registros duplicados

In [92]:
# Se crearán filas duplicadas ya que al desanidar hay dos datos en una fila, Ejemplo: 'genre', donde algunos tienen 'Animation, Comedy', estos valores se desagregarán.

# Se crean nuevas columnas 
movies['collection_names'] = movies['collection_name'].str.split(', ')
movies['genres'] = movies['genre'].str.split(', ')
movies['companies'] = movies['company'].str.split(', ')
movies['countries'] = movies['country'].str.split(', ')
movies['languages'] = movies['spoken_language'].str.split(', ')

# Se duplican los registros
movies=movies.explode('collection_names').reset_index(drop=True)
movies=movies.explode('genres').reset_index(drop=True)
movies=movies.explode('companies').reset_index(drop=True)
movies=movies.explode('countries').reset_index(drop=True)
movies=movies.explode('languages').reset_index(drop=True)

#Se eliminan las columnas originales
movies.drop('collection_name',axis=1,inplace=True)
movies.drop('genre',axis=1,inplace=True)
movies.drop('company',axis=1,inplace=True)
movies.drop('country',axis=1,inplace=True)
movies.drop('spoken_language',axis=1,inplace=True)

In [93]:
# Comparamos con el anterior código para ver si hubieron cambios
print (movies.collection_names.unique())
print(movies.genres.unique())
print(movies.companies.unique())
print(movies.countries.unique())
print(movies.languages.unique())

['Toy Story Collection' 'None' 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']
['Animation' 'Comedy' 'Family' 'Adventure' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'History' 'Science Fiction'
 'Mystery' 'War' 'Foreign' 'None' 'Music' 'Documentary' 'Western'
 'TV Movie' 'Carousel Productions' 'Vision View Entertainment'
 'Telescene Film Group Productions' 'Aniplex' 'GoHands' 'BROSTA TV'
 'Mardock Scramble Production Committee' 'Sentai Filmworks'
 'Odyssey Media' 'Pulser Productions' 'Rogue State' 'The Cartel']
['Pixar Animation Studios' 'TriStar Pictures' 'Teitler Film' ...
 'Pirie Productions' 'Sine Olivia' 'Yermoliev']
['United States of America' 'Germany' 'United Kingdom' 'France' 'Italy'
 'Spain' 'China' 'Australia' 'South Africa' 'Canada' 'None' 'Switzerland'
 'Belgium' 'Japan' 'Iran' 'Netherlands' 'Hong Kong' 'Tunisia' 'Ireland'
 'Dominican Republic' 'Croatia' 'Russia' 'Macedonia' 'Austria' 'Taiwan'
 'Ne

In [94]:
print ('Cantidad de collection_names:',movies.collection_names.nunique())
print('Cantidad de genres:',movies.genres.nunique())
print('Cantidad de companies:',movies.companies.nunique())
print('Cantidad de countries:',movies.countries.nunique())
print('Cantidad de languages:',movies.languages.nunique())

Cantidad de collection_names: 1721
Cantidad de genres: 33
Cantidad de companies: 23567
Cantidad de countries: 161
Cantidad de languages: 76


In [95]:
# Observamos que las filas se duplicaron y se desagregaron los datos separados por comas.
movies.head(10)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_names,genres,companies,countries,languages
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,TriStar Pictures,United States of America,English
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,TriStar Pictures,United States of America,Français
5,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Teitler Film,United States of America,English
6,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Teitler Film,United States of America,Français
7,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Interscope Communications,United States of America,English
8,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Adventure,Interscope Communications,United States of America,Français
9,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,Fantasy,TriStar Pictures,United States of America,English


## 5. Valores Nulos

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [96]:
# Rellenamos los valores faltantes en 'revenue' y 'budget' con el valor 0.
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

# Contamos la cantidad de valores nulos en 'revenue' y 'budget'
print('Valores nulos en revenue',movies['revenue'].isna().sum())
print('Valores nulos en budget',movies['revenue'].isna().sum())

Valores nulos en revenue 0
Valores nulos en budget 0


Los valores nulos del campo release date deben eliminarse.

In [97]:
# Eliminamos las filas con valores nulos de 'realease_date'
movies=movies.dropna(subset=['release_date'])

# Contamos la cantidad de valores nulos en 'realease_date'
print('Valores nulos en realease_date:', movies['release_date'].isna().sum())

Valores nulos en realease_date: 0


Los valores en la columna "title" del dataframe "movies" cuentan con vacíos

In [98]:
# Observamos la cantidad de valores en la columna "title"
movies['title'].info()

<class 'pandas.core.series.Series'>
Index: 422016 entries, 0 to 422194
Series name: title
Non-Null Count   Dtype 
--------------   ----- 
422004 non-null  object
dtypes: object(1)
memory usage: 6.4+ MB


In [99]:
# Aplicamos una máscara para verificar valores vacíos
mask = movies['title'] > ''
movies = movies.loc[mask]

In [100]:
# Observamos que se eliminaron algunos valores vacíos
movies['title'].info()

<class 'pandas.core.series.Series'>
Index: 422004 entries, 0 to 422194
Series name: title
Non-Null Count   Dtype 
--------------   ----- 
422004 non-null  object
dtypes: object(1)
memory usage: 6.4+ MB


## 6. Modificación de fecha

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [101]:
# Transformamos 'release_date' en el formato 'aaaa-mm-dd', en caso el formato sea incorrecto.
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Creamos la columna 'release_year' extrayendo el año de 'release_date'.
movies['release_year'] = pd.to_datetime(movies['release_date'], errors='coerce').dt.year
movies['release_year'] = movies['release_year'].astype('Int64')

In [102]:
movies[['release_date','release_year']]

,release_date,release_year
0,1995-10-30,1995
1,1995-10-30,1995
2,1995-10-30,1995
3,1995-12-15,1995
4,1995-12-15,1995
...,...,...
422190,2003-08-01,2003
422191,2003-08-01,2003
422192,2003-08-01,2003
422193,1917-10-21,1917


In [103]:
# Observamos que la columna "release_date" del dataframe "movies" es una fecha pero está en formato objeto.
movies['release_date'].info()

<class 'pandas.core.series.Series'>
Index: 422004 entries, 0 to 422194
Series name: release_date
Non-Null Count   Dtype 
--------------   ----- 
422004 non-null  object
dtypes: object(1)
memory usage: 6.4+ MB


In [104]:
# Convertimos la columna "release_date" del dataframe "movies" en formato fecha.
movies['release_date'] = pd.to_datetime(movies['release_date'])

In [105]:
# Observamos que la columna "release_date" del dataframe "movies" está en formato fecha.
movies['release_date'].info()

<class 'pandas.core.series.Series'>
Index: 422004 entries, 0 to 422194
Series name: release_date
Non-Null Count   Dtype         
--------------   -----         
422004 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 6.4 MB


In [106]:
# Borramos nulos de 'release_date' y 'release_year'
movies=movies.dropna(subset=['release_date'])
movies=movies.dropna(subset=['release_year'])

# Contamos la cantidad de valores nulos en 'realease_date' y 'release_year'
print('Valores nulos en realease_date:', movies['release_date'].isna().sum())
print('Valores nulos en realease_year:', movies['release_year'].isna().sum())

Valores nulos en realease_date: 0
Valores nulos en realease_year: 0


In [107]:
# Observamos la cantidad de nulos del dataframe "movies"
movies.isna().sum()

budget                    0
id                        0
original_language        96
overview               2495
popularity                0
release_date              0
revenue                   0
runtime                 544
status                  143
tagline              193173
title                     0
vote_average              0
vote_count                0
collection_names          0
genres                    0
companies                 0
countries                 0
languages                 0
release_year              0
dtype: int64

In [108]:
# Observamos la cantidad de nulos del dataframe "creditos"
creditos.isna().sum()

id            0
Director    887
dtype: int64

## 7. Creación de columna retorno de inversión

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [109]:
# Convertimos 'revenue' y 'budget' a tipo float
movies['revenue'] = pd.to_numeric(movies['revenue'])
movies['budget'] = pd.to_numeric(movies['budget'])

# Calculamos el retorno de inversión
movies['return'] = movies['revenue'].divide(movies['budget']).replace([np.inf, -np.inf],0).fillna(0)
movies[['return']]


,return
0,12.451801
1,12.451801
2,12.451801
3,4.043035
4,4.043035
...,...
422190,0.000000
422191,0.000000
422192,0.000000
422193,0.000000


## 8. Unimos el dataframe "creditos" con "movies"

In [110]:
# Tienen en común id.
movies.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_names,genres,companies,countries,languages,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995,12.451801
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995,12.451801


In [111]:
# Tienen en común id.
creditos.head(1)

,id,Director
0,862,John Lasseter


In [112]:
# Convertimos la columna 'id' de "creditos" y "movies" a tipo int64 para que coincidan entre ambos.
creditos['id'] = creditos['id'].astype('int64')
movies['id'] = movies['id'].astype('int64')

# Unimos los dataframes en función de la columna 'id', creando un nuevo dataframe "datos".
datos = pd.merge(movies, creditos, on='id')


In [113]:
# Observamos que los datos se hayan unido.
datos.sample(3)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,collection_names,genres,companies,countries,languages,release_year,return,Director
50295,100000,11713,zh,Chen Zhen returns to the international compoun...,9.523138,1972-03-22,0.0,108.0,Released,Bruce Lee has done the impossible... ...HE'S S...,...,7.4,164.0,None,Action,Golden Harvest Company,Hong Kong,普通话,1972,0.0,Lo Wei
54199,0,9028,it,A mute gunslinger fights in the defense of a g...,3.237317,1968-02-21,0.0,100.0,Released,His voice was the silence of death!,...,7.5,58.0,None,Western,Les Films Corona,France,Italiano,1968,0.0,Sergio Corbucci
401508,0,300438,et,Young and active nationalist Aleksander Keskül...,0.33279,1997-10-02,0.0,99.0,Released,NaN,...,6.0,1.0,None,Thriller,Faama Film,Russia,English,1997,0.0,Hardi Volmer


In [114]:
# Observamps los datos vacíos del nuevo datarframe "datos".
datos.isna().sum()

budget                    0
id                        0
original_language        96
overview               2495
popularity                0
release_date              0
revenue                   0
runtime                 544
status                  143
tagline              195330
title                     0
vote_average              0
vote_count                0
collection_names          0
genres                    0
companies                 0
countries                 0
languages                 0
release_year              0
return                    0
Director               1512
dtype: int64

In [115]:
# Cambiamos los datos nulos del dataframe por 'None'.
datos = datos.where(pd.notna(datos), 'None')

In [116]:
# Verificamos que los datos estén parejos.
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424431 entries, 0 to 424430
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   budget             424431 non-null  int64         
 1   id                 424431 non-null  int64         
 2   original_language  424431 non-null  object        
 3   overview           424431 non-null  object        
 4   popularity         424431 non-null  object        
 5   release_date       424431 non-null  datetime64[ns]
 6   revenue            424431 non-null  float64       
 7   runtime            424431 non-null  object        
 8   status             424431 non-null  object        
 9   tagline            424431 non-null  object        
 10  title              424431 non-null  object        
 11  vote_average       424431 non-null  float64       
 12  vote_count         424431 non-null  float64       
 13  collection_names   424431 non-null  object  

## 9. Exportamos el dataframe "datos" a un archivo .csv

In [117]:
# Obtenemos el archivo con el ETL aplicado
datos.to_csv("datos_finales.csv", index=False)

## 10. Realizamos un dataset con los valores necesarios para las consultas en la API y el Modelo de recomendación

In [118]:
# Realizaremos un dataset con los valores necesarios ya que el peso del último dataset exportado es muy grande.
# Leemos el dataset "datos".
datos = pd.read_csv('datos_finales.csv', sep=',')

In [119]:
datos.head(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_average,vote_count,collection_names,genres,companies,countries,languages,release_year,return,Director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995,12.451801,John Lasseter
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995,12.451801,John Lasseter


In [120]:
columnas=['id','budget','original_language','popularity','release_date','revenue','runtime','title', 'vote_average', 'vote_count', 'collection_names', 'genres', 'companies', 'countries', 'languages', 'release_year', 'return', 'Director' ]

In [121]:
data_movies= datos[columnas]

In [122]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424431 entries, 0 to 424430
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 424431 non-null  int64  
 1   budget             424431 non-null  int64  
 2   original_language  424335 non-null  object 
 3   popularity         424431 non-null  float64
 4   release_date       424431 non-null  object 
 5   revenue            424431 non-null  float64
 6   runtime            423887 non-null  float64
 7   title              424431 non-null  object 
 8   vote_average       424431 non-null  float64
 9   vote_count         424431 non-null  float64
 10  collection_names   55238 non-null   object 
 11  genres             421287 non-null  object 
 12  companies          397322 non-null  object 
 13  countries          413069 non-null  object 
 14  languages          411960 non-null  object 
 15  release_year       424431 non-null  int64  
 16  re

In [123]:
data_movies.to_csv("datos_main.csv", index=False)